In [1]:
import pandas as pd
import logging

# Log fayl manzili
log_path = r"C:\Users\Rasulbek907\Desktop\Project_8\Log\feature_selection.log"

# Log sozlamalari
logging.basicConfig(
    filename=log_path,
    filemode='a',  # Append mode
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

try:
    logging.info("CSV fayl o'qilmoqda:...")
    df = pd.read_csv(r"C:\Users\Rasulbek907\Desktop\Project_8\Data\Preprocessing\Preprosessed.csv")
    logging.info(f"Fayl muvaffaqiyatli o'qildi. Satırlar soni: {len(df)} ustunlar soni: {len(df.columns)}")
except Exception as e:
    logging.error(f"CSV faylni o'qishda xatolik: {e}")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0              371528 non-null  int64  
 1   index                   371528 non-null  float64
 2   dateCrawled             371528 non-null  float64
 3   name                    371528 non-null  float64
 4   price                   371528 non-null  float64
 5   vehicleType             371528 non-null  float64
 6   yearOfRegistration      371528 non-null  float64
 7   powerPS                 371528 non-null  float64
 8   model                   371528 non-null  float64
 9   kilometer               371528 non-null  float64
 10  monthOfRegistration     371528 non-null  float64
 11  fuelType                371528 non-null  float64
 12  brand                   371528 non-null  float64
 13  dateCreated             371528 non-null  float64
 14  nrOfPictures        

In [13]:
import pandas as pd
import plotly.express as px

# Raqamli ustunlarni tanlash
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

# faqat raqamli ustunlar bo'yicha korrelyatsiya
corr_matrix = df[numeric_cols].corr()

# kilometer ustunining boshqa ustunlar bilan korrelyatsiyasi
life_span_corr = corr_matrix[['kilometer']].sort_values(by='kilometer', ascending=False)

# Heatmap uchun DataFrame ni transformatsiya qilish
heatmap_data = life_span_corr.reset_index()
heatmap_data.columns = ['Feature', 'Correlation_with_life_span']

# Plotly Express yordamida bar grafik
fig = px.bar(
    heatmap_data,
    x='Feature',
    y='Correlation_with_life_span',
    title='kilometer bilan boshqa raqamli ustunlar o\'rtasidagi korrelyatsiya',
    labels={'Correlation_with_life_span': 'Korrelyatsiya qiymati', 'Feature': 'Xususiyatlar'}
)

fig.show()


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [4]:
import sys
print(sys.executable)


c:\Users\Rasulbek907\Desktop\Project_8\mpvenv\Scripts\python.exe


In [ ]:
import numpy as np
import plotly.figure_factory as ff

# Qiymatlarni 2 xonagacha qisqartirish
annot_text = np.around(corr_matrix.values, decimals=2).astype(str)

fig = ff.create_annotated_heatmap(
    z=corr_matrix.values,
    x=list(corr_matrix.columns),
    y=list(corr_matrix.index),
    annotation_text=annot_text,
    colorscale='Viridis',
    showscale=True
)

fig.update_layout(title='Raqamli ustunlar o\'rtasidagi korrelyatsiya matritsasi')
fig.show()

# Feature Selection Filtering 

In [ ]:
import seaborn as sns               
import matplotlib.pyplot as plt     


corr_matrix = df.corr().abs()  

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))  


to_drop = [col for col in upper.columns if any(upper[col] > 0.8)]  
corr_matrix = df.corr().abs() 
print(corr_matrix)  


In [ ]:

import numpy as np   

corr_matrix = df.corr().abs()  

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))  

to_drop = [col for col in upper.columns if any(upper[col] > 0.95)]  

df_filtered = df.drop(columns=to_drop)  
print("\nOriginal shape:", df.shape)  

print("Shape after dropping correlated features:", df_filtered.shape) 


In [ ]:
df_filtered.info()

In [ ]:
# Yuqori korrelyatsiyaga ega juftliklarni saqlash uchun bo‘sh ro‘yxat yaratamiz
high_corr_pairs = []


for col in upper.columns:
    
    for row in upper.index:
        
        if upper.loc[row, col] is not np.nan and upper.loc[row, col] > 0.95:
            
            high_corr_pairs.append([row, col, round(upper.loc[row, col], 2)])


corr_table = pd.DataFrame(high_corr_pairs, columns=['Feature 1', 'Feature 2', 'Correlation'])

print("Highly correlated feature pairs (correlation > 0.95):")
print(corr_table)

In [ ]:
import plotly.express as px  

corr_long = corr_matrix.reset_index().melt(id_vars='index')  

corr_long.columns = ['Feature 1', 'Feature 2', 'Correlation']  

fig = px.imshow(
    corr_matrix,                  
    text_auto='.2f',              
    aspect="auto",                
    color_continuous_scale='RdBu_r',  
    zmin=-1, zmax=1,             
    title="Correlation Matrix Heatmap (Interactive)"  
)


fig.update_layout(
    width=900,          
    height=800,          
    xaxis_title="Features",  
    yaxis_title="Features"   
)

fig.show()

# Low Variance

In [ ]:
# Low Variance
from sklearn.feature_selection import VarianceThreshold  
threshold = 0.01  
selector = VarianceThreshold(threshold=threshold)  
selector.fit(df_filtered)  
low_variance_features = df_filtered.columns[~selector.get_support()] 
df_low_variance_filtered = df_filtered.drop(columns=low_variance_features)  
print("\nOriginal shape:", df_filtered.shape)  
print("Shape after dropping low-variance features:", df_low_variance_filtered.shape)  

In [ ]:
# Past variansli ustunlarni aniqlash:

low_variance_features = df_filtered.columns[~selector.get_support()]
print("Features with low variance (to drop):", list(low_variance_features))

In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns
variances = df[numeric_cols].var()
threshold = 0.01
low_variance_features = variances[variances < threshold].index.tolist()
var_df = pd.DataFrame({
    'Feature': variances.index,
    'Variance': variances.values,
    'LowVariance': ['Yes' if f in low_variance_features else 'No' for f in variances.index]
})


fig = px.bar(
    var_df,
    x='Feature',        
    y='Variance',       
    color='LowVariance',  
    color_discrete_map={'Yes': 'red', 'No': 'blue'},  
    text='Variance',    
    title='Feature Variance (Low-Variance Highlighted in Red)'  
)


fig.update_layout(
    xaxis_tickangle=-45,
    width=1000,
    height=600
)

fig.show()

In [ ]:
df_filtered.info()

In [ ]:
df_filtered.to_csv(r"C:\Users\Rasulbek907\Desktop\Final_Project\Data\Feature_selection\Filtered_Features.csv")